In [2]:
import os
currentPath = os.getcwd()

import configparser
settings = configparser.ConfigParser()
settings._interpolation = configparser.ExtendedInterpolation()
settings.read(currentPath + "/config/main_config.ini")

libs_path = settings.get('Main-Config', 'libs')
output_path = settings.get('Main-Config', 'output_path') + "/rotated_subset"

subsetSizeX = int(settings.get('Main-Config', 'subsetSizeX'))
subsetSizeY = int(settings.get('Main-Config', 'subsetSizeY'))
marginX = int(settings.get('Main-Config', 'marginX'))
marginY = int(settings.get('Main-Config', 'marginY'))

import sys
sys.path.append(libs_path)

from osgeo import gdal,ogr, osr
gdal.UseExceptions()
ogr.UseExceptions()
osr.UseExceptions()

import snappy
from snappy import ProductIO
from snappy import HashMap
from snappy import GPF
from snappy import jpy

import os
import os.path
from collections import OrderedDict
from datetime import datetime
import datetime
from datetime import timedelta, date
import time

import numpy as np
import glob
from itertools import cycle
import math
import shutil
import json
import geojson
import random

import array
import numpy as np
import itertools
import matplotlib.pyplot as plt

# https://stackoverflow.com/questions/24816237/ipython-notebook-clear-cell-output-in-code
from IPython.display import clear_output

outFormat = 'ESRI Shapefile'

GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()
HashMap = snappy.jpy.get_type('java.util.HashMap')

System = jpy.get_type('java.lang.System')
System.gc()

PrintWriterProgressMonitor = jpy.get_type('com.bc.ceres.core.PrintWriterProgressMonitor')
FileOutputStream = jpy.get_type('java.io.FileOutputStream')
ProductData = jpy.get_type('org.esa.snap.core.datamodel.ProductData')
Product = jpy.get_type('org.esa.snap.core.datamodel.Product')
GeoPos = jpy.get_type('org.esa.snap.core.datamodel.GeoPos')
PixelPos = jpy.get_type('org.esa.snap.core.datamodel.PixelPos')

from keras.datasets import mnist
from keras.models import Model
from keras.layers.core import Dense, Dropout, Flatten
#from tensorflow.keras.optimizers import RMSprop
from keras.utils import np_utils
from keras.layers.convolutional import Conv2D, MaxPooling2D, UpSampling2D
from keras.layers import Input, BatchNormalization
from sklearn import metrics as me
from scipy import stats
from keras import models
from keras import layers
from sklearn.preprocessing import Normalizer

import tensorflow as tf

folderStart = "K:/TM/data_output/subset_start"
folderResult = "K:/TM/data_output/subset_rotated"

Using TensorFlow backend.
C:\Users\Vincent\anaconda3\envs\python36_\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Vincent\anaconda3\envs\python36_\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Vincent\anaconda3\envs\python36_\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Vincent\anaconda3\en

In [2]:
def getProductList(path, search):
    
    #products = []
    products = []
    
    lbs_input_path = os.path.join(path, search)
    files = glob.glob(lbs_input_path)

    for f in files:
        temp = np.load(f)
        products.append(temp['arr_0'])
        
    return products
    
def getProductList2(path, search): #TEMP
    
    products = []
        
    for pathF in os.listdir(path):
        
        for pathF2 in os.listdir(path + "/" + pathF):
        
            if search in pathF2:
                temp = np.load(path + "/" + pathF + "/" + pathF2)
                products.append(temp['arr_0'])
            
    return products

def getProductFileList(path, search):
    
    lbs_input_path = os.path.join(path, search)
    files = glob.glob(lbs_input_path)

    return files

In [ ]:
def reduceData(data):
    
    return None

In [3]:
subsetStarts = getProductList(folderStart, '*.npz')
Y = getProductList2(folderResult, 'VV.npz')

In [ ]:
print(len(subsetStarts))
print(len(Y))

print(len(subsetStarts[0]))
print(len(subsetStarts[0][0]))
print(len(subsetStarts[0][0][0]))

print(len(Y[0]))
print(len(Y[0][0]))

print(type(subsetStarts))
print(type(Y))

In [4]:
X = []

for x_ in subsetStarts:
    
    X.append(x_[1:])

del subsetStarts

In [ ]:
print(len(X))
print(len(X[0]))
print(len(Y))

In [5]:
X_trainL = X[:80000] 
Y_trainL = Y[:80000]

X_testL = X[80000:100000]
Y_testL = Y[80000:100000]

del X
del Y

print(len(X_trainL))
print(len(Y_trainL))
print(len(X_testL))
print(len(Y_testL))

80000
80000
20000
20000


In [ ]:
print(len(X_train))
print(len(X_train[0]))
print(len(X_train[0][0]))
print(len(X_train[0][0][0]))

In [ ]:
print(len(Y_train))
print(len(Y_train[0]))
print(len(Y_train[0][0]))
print(len(Y_train[0][0][0]))

In [6]:
print(type(X_trainL))
print(type(Y_trainL))
print(type(X_testL))
print(type(Y_testL))

X_train = np.array(X_trainL)
del X_trainL

Y_train = np.array(Y_trainL)
del Y_trainL

X_test = np.array(X_testL)
del X_testL

Y_test = np.array(Y_testL)
del Y_testL

print(type(X_train))
print(type(Y_train))
print(type(X_test))
print(type(Y_test))

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:
print(X_train[0][0][0][0])
print(X_train[0][1][0][0])
print(X_train[0][2][0][0])
print(X_train[0][3][0][0])
print(X_train[0][4][0][0])
print(X_train[0][5][0][0])
print(X_train[0][6][0][0])

In [7]:
print(X_train.shape)

X_train = np.swapaxes(X_train, 1, 2)

print(X_train.shape)

X_train = np.swapaxes(X_train, 2, 3)

print(X_train.shape)

X_test = np.swapaxes(X_test, 1, 2)
X_test = np.swapaxes(X_test, 2, 3)

print(X_test.shape)

(80000, 7, 50, 50)
(80000, 50, 7, 50)
(80000, 50, 50, 7)
(20000, 50, 50, 7)


In [ ]:
print(X_train[0][0][0][0])
print(X_train[0][0][0][1])
print(X_train[0][0][0][2])
print(X_train[0][0][0][3])
print(X_train[0][0][0][4])
print(X_train[0][0][0][5])
print(X_train[0][0][0][6])

print(X_train[0][0][0])

In [ ]:
print(Y_train.shape)
print(Y_test.shape)

In [ ]:
print(Y_train[0][0][0])
print(Y_train[0][0][1])
print(Y_train[0][1][0])
print(Y_train[0][54][78])
print(Y_train[0][120][120])
print(Y_train[0][450][40])
print(Y_train[0][499][450])
print(Y_train[0][120][430])
print(Y_train[0][40][70])
print(Y_train[0][405][60])
print(Y_train[0][40][230])

In [8]:
Y_train = Y_train.reshape(len(Y_train), 50, 50, 1).astype('float64')
Y_test = Y_test.reshape(len(Y_test), 50, 50, 1).astype('float64')

In [ ]:
print(Y_train[0][0][0])
print(Y_train[0][0][1])
print(Y_train[0][1][0])
print(Y_train[0][54][78])
print(Y_train[0][120][120])
print(Y_train[0][450][40])
print(Y_train[0][499][450])
print(Y_train[0][120][430])
print(Y_train[0][40][70])
print(Y_train[0][405][60])
print(Y_train[0][40][230])

In [ ]:
Y_train_max = np.amax(Y_train)
Y_train_min = np.amin(Y_train)

print(Y_train_max)
print(Y_train_min)

Y_train = (Y_train - Y_train_min) / (Y_train_max - Y_train_min + 0.00000001)

print(np.amax(Y_train))
print(np.amin(Y_train))

In [9]:
#l0 = Input(shape=(height, width, 9), name='l0')

#l0 = Input(shape=(len(subsetResults[0]), len(subsetResults[0][0]), 7), name='l0')

l0 = Input(shape=(50, 50, 7), name='l0')
l0_n = BatchNormalization()(l0)

l1 = Conv2D(7, (5, 5), padding='same', activation='relu', name='l1')(l0_n)
l1_mp = MaxPooling2D((2, 2), name='l1_mp')(l1)

l2 = Conv2D(7, (3, 3), padding='same', activation='relu', name='l2')(l1_mp)
#l2_mp = MaxPooling2D((5, 5), name='l2_mp')(l2)

#l22 = Conv2D(14, (5, 5), padding='same', activation='relu', name='l22')(l2)

l3 = Conv2D(7, (5, 5), padding='same', activation='relu', name='l3')(l2)
l3_us = UpSampling2D((2, 2))(l3)

l4 = Conv2D(1, (5, 5), padding='same', activation='relu', name='l4')(l3_us)
l4_us = UpSampling2D((2, 2))(l4)

l45 = Conv2D(1, (3, 3), padding='same', activation='relu', name='l45')(l4_us)
l45_mp = MaxPooling2D(pool_size=(2, 2), name='l45_mp')(l45)

l5 = Conv2D(1, (5, 5), padding='same', activation='relu', name='l5')(l45_mp)

#flat = Flatten(name='flat')(l4_us)

#l5 = Dense((250, 250), activation='relu', name='l5')(flat)

#l5 = Dense(n_classes, activation='softmax', name='l5')(l4)

model = Model(inputs=l0, outputs=l5)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
l0 (InputLayer)              (None, 50, 50, 7)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 50, 50, 7)         28        
_________________________________________________________________
l1 (Conv2D)                  (None, 50, 50, 7)         1232      
_________________________________________________________________
l1_mp (MaxPooling2D)         (None, 25, 25, 7)         0         
_________________________________________________________________
l2 (Conv2D)                  (None, 25, 25, 7)         448       
_________________________________________________________________
l3 (Conv2D)                  (None, 25, 25, 7)         1232      
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 50, 50, 7)         0   

In [10]:
batch_size = 1024
n_epoch = 100

#model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])
model.compile(loss='mean_absolute_percentage_error', optimizer='RMSprop', metrics=['mean_absolute_percentage_error'])
history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=n_epoch, verbose=1, validation_data=(X_test, Y_test))

Train on 80000 samples, validate on 20000 samples
Epoch 1/100
80000/80000 [==============================] - 392s 5ms/step - loss: 97.8939 - mean_absolute_percentage_error: 97.8939 - val_loss: 236.3965 - val_mean_absolute_percentage_error: 236.3966
Epoch 2/100
80000/80000 [==============================] - 391s 5ms/step - loss: 51.5712 - mean_absolute_percentage_error: 51.5712 - val_loss: 71.3933 - val_mean_absolute_percentage_error: 71.3933
Epoch 3/100
80000/80000 [==============================] - 390s 5ms/step - loss: 43.6593 - mean_absolute_percentage_error: 43.6593 - val_loss: 59.3625 - val_mean_absolute_percentage_error: 59.3625
Epoch 4/100
80000/80000 [==============================] - 392s 5ms/step - loss: 41.4839 - mean_absolute_percentage_error: 41.4839 - val_loss: 59.3807 - val_mean_absolute_percentage_error: 59.3807
Epoch 5/100
80000/80000 [==============================] - 391s 5ms/step - loss: 39.9419 - mean_absolute_percentage_error: 39.9419 - val_loss: 56.0188 - val_mea

KeyboardInterrupt: 

In [ ]:
temp = X_train[0]

print(temp.shape)

temp = np.swapaxes(temp, 1, 2)

print(temp.shape)

temp = np.swapaxes(temp, 0, 1)

print(temp.shape)

In [ ]:
for band in temp:
    
    print(np.amin(band))
    print(np.amax(band))
    print("--------------")

tempR = temp.reshape(7, 250000).astype('float64')
    
bins = []
step = 0.01

for i in range(0, 55):
    bins.append(step * i * 1.0)

plt.hist(tempR[0])  # arguments are passed to np.histogram
plt.title("X_train[0][0] - Histogram")

plt.show()

In [ ]:
temp = Y_train[0]

temp = temp.reshape(250000).astype('float64')

print(np.amin(temp))
print(np.amax(temp))

bins = []
step = 0.02

for i in range(0, 45):
    bins.append(step * i * 1.0)

plt.hist(temp, bins=bins)  # arguments are passed to np.histogram
plt.title("Y_train[0] - Histogram")

plt.show()

In [ ]:
# Search Min diff

temp = Y_train[0]
temp = temp.reshape(250000).astype('float64')

minDiff = np.amax(temp)

for i in range(1, len(temp)):
    
    tempDiff = abs(temp[i - 1] - temp[i])
    
    if tempDiff != 0 and tempDiff < minDiff :
        minDiff = tempDiff
        
print(minDiff)

In [ ]:
temp = Y_train
temp = temp.reshape(150000000).astype('float64')

print(np.amax(temp))

bins = []
step = 0.02

for i in range(0, 45):
    bins.append(step * i * 1.0)

plt.hist(temp, bins=bins)  # arguments are passed to np.histogram
plt.title("Y_train[0] - Histogram")

plt.show()

In [ ]:
print(temp[1])
print(type(temp[0]))

In [ ]:
print(np.amax(Y_train))
print(np.amin(Y_train))

In [ ]:
Y_train = Y_train.round(7)

In [ ]:
print(np.amax(Y_train))
print(np.amin(Y_train))

In [ ]:
plt.plot(history.history['loss'], label='Training')
plt.plot(history.history['val_loss'], label='Testing')
plt.legend()
plt.grid()

score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])